In [1]:
import re

In [4]:
content = "Hello World"

In [6]:
print(re.search("W",content)) #search : 문자열 전체 검색

<_sre.SRE_Match object; span=(6, 7), match='W'>


In [32]:
print(re.match("W", content)) #문자열 처음부터 검색

None


In [33]:
data = """
park 800905-1049118
kim 700905-1059119
"""

In [35]:
result=[]
for line in data.split("\n"):
    word_result=[]
    for word in line.split(" "):
        if len(word) == 14 and word[:6].isdigit() and word[7:].isdigit():
            word = word[:6] + "-" + "*******"
        word_result.append(word)
    result.append(" ".join(word_result))
print("\n".join(result))


park 800905-*******
kim 700905-*******



In [44]:
pat = re.compile("(\d{6}_[-]\d{7})")
print(pat.sub("\g<1>-*******",data))


park 800905-1049118
kim 700905-1059119



In [50]:
p = re.compile('Crow|Servo')
m = p.match('CrowHello')
print(m)

<_sre.SRE_Match object; span=(0, 4), match='Crow'>


In [53]:
print(re.search('^Life','Life is too short'))
print(re.search('^Life','My Life'))

<_sre.SRE_Match object; span=(0, 4), match='Life'>
None


In [55]:
print(re.search('short$', 'Life is too short'))
print(re.search('short$', 'Life is too short, you need python'))

<_sre.SRE_Match object; span=(12, 17), match='short'>
None


In [56]:
p = re.compile('(ABC)+')

In [57]:
m = p.search('ABCABCABC OK?')

In [58]:
print(m.group())

ABCABCABC


In [59]:
p = re.compile(r'\bclass\b')

In [63]:
print(p.search('no class at all'))
print(p.search('one subclass is'))
print(p.search('the declassified algorithm'))

<_sre.SRE_Match object; span=(3, 8), match='class'>
None
None


In [87]:
p = re.compile(r"\w+\w+\d+[-]\d+[-]\d+")
m = p.search("park 010-1234-1234")
print(m.group())

010-1234-1234


In [91]:
p = re.compile("r(\w+)\s+\d+[-]\d+[-]\d+") # r : raw string.. 백슬래시 살려둠 파이썬은 원래 생략시킨단다

In [92]:
m = p.search("park 010-1234-1234")

In [93]:
print(m.group(1))

k


In [94]:
data = "정규식 너무 싫다"

In [100]:
p = re.compile(r"\W")

In [101]:
re.split(p, data)

['정규식', '너무', '싫다']